In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pip install keras==2.1.5

In [ ]:
pip install keras-self-attention

In [ ]:
!pip install emoji

In [ ]:
!pip install ekphrasis

In [7]:
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
import re
import os
from collections import Counter

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [ ]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=True,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

In [9]:
def print_text(texts,i,j):
    for u in range(i,j):
        print(texts[u])
        print()

In [10]:
df_1 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2016train-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_1.head(5)) #last N rows
# print(len(df_1))

df_2 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2016test-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_2.head(5)) #last N rows
# print(len(df_2))

df_3 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2016devtest-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_3.head(5)) #last N rows
# print(len(df_3))

df_4 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2016dev-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_4.head(5)) #last N rows
# print(len(df_4))

df_5 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2015train-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_5.head(5)) #last N rows
# print(len(df_5))

df_6 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2015test-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_6.head(5)) #last N rows
# print(len(df_6))

df_7 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2014test-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_7.head(5)) #last N rows
# print(len(df_7))

df_8 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2014sarcasm-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_8.head(5)) #last N rows
# print(len(df_8))

df_9 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2013train-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_9.head(5)) #last N rows
# print(len(df_9))

df_10 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2013test-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_10.head(5)) #last N rows
# print(len(df_10))

df_11 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2013dev-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_11.head(5)) #last N rows
# print(len(df_11))


<h2>Balancing the data</h2>

In [ ]:
df = pd.DataFrame()
df = df.append(df_1, ignore_index = True)
df = df.append(df_2, ignore_index = True)
df = df.append(df_3, ignore_index = True)
df = df.append(df_4, ignore_index = True)

df = df.append(df_5, ignore_index = True)
df = df.append(df_6, ignore_index = True)
df = df.append(df_7, ignore_index = True)
df = df.append(df_8, ignore_index = True)

df = df.append(df_9, ignore_index = True)
df = df.append(df_10, ignore_index = True)
df = df.append(df_11, ignore_index = True)

print(df.head(5))
print(len(df))

In [12]:
# Testing for null values
# lol = np.asarray(df_[1].isnull())

# for i in range(0,len(lol)):
#     if lol[i]:
#         print(i)

In [ ]:
print(len(df))

In [ ]:
text_array = df[2]
labels = df[1]
print("Length of training data: ",len(text_array))
print_text(text_array,0,10)

In [ ]:
df_val = pd.read_csv('/content/drive/My Drive/Semeval 2017/Test/SemEval2017-task4-test.subtask-A.english.txt', delimiter='\n', encoding='utf-8', header=None)
print(df_val.tail(5)) #last N rows
print(len(df_val))

In [16]:
lol = []
test_set = np.asarray(df_val[0])
for i in range(0,len(df_val)):
    temp = np.asarray(test_set[i].split("\t"))
    temp = temp.reshape((3))
    lol.append(temp)

In [ ]:
df_val = pd.DataFrame(lol)
df_val.head(5)

In [ ]:
text_array_val = df_val[2]
labels_val = df_val[1]
print("Length of validation data: ",len(text_array_val))
print_text(text_array_val,0,10)

In [ ]:
print(Counter(labels))
print(Counter(labels_val))

In [ ]:
#removing website names
def remove_website(text):
    return " ".join([word if re.search("r'https?://\S+|www\.\S+'|((?i).com$|.co|.net)",word)==None else "" for word in text.split(" ") ])

# Training set 
text_array = text_array.apply(lambda text: remove_website(text))
print_text(text_array,0,10)

print("**************************************************************************")

# Validation set 
text_array_val = text_array_val.apply(lambda text: remove_website(text))
print_text(text_array_val,0,10)

In [21]:
# Functions for chat word conversion
f = open("/content/drive/My Drive/slang.txt", "r")
chat_words_str = f.read()
chat_words_map_dict = {}
chat_words_list = []

for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [ ]:
# Chat word conversion
# Training set
text_array = text_array.apply(lambda text: chat_words_conversion(text))
print_text(text_array,0,10)

print("********************************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: chat_words_conversion(text))
print_text(text_array_val,0,10)

In [ ]:
#Function for emoticon conversion
from emoticons import EMOTICONS

def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', " ".join(EMOTICONS[emot].replace(",","").split()), text)
    return text


#testing the emoticon function
text = "Hello :-) :-)"
text = convert_emoticons(text)
print(text + "\n")

In [ ]:
# Emoticon conversion
# Training set
text_array = text_array.apply(lambda text: convert_emoticons(text))
print_text(text_array,0,10)

print("**********************************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: convert_emoticons(text))
print_text(text_array_val,0,10)

In [ ]:
# FUnction for removal of emoji
import emoji

def convert_emojis(text):
    text = emoji.demojize(text, delimiters=(" ", " "))
    text = re.sub("_|-"," ",text)
    return text

# Training set
text_array = text_array.apply(lambda text: convert_emojis(text))
print_text(text_array,0,10)

print("**************************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: convert_emojis(text))
print_text(text_array_val,0,10)

In [ ]:
# Ekphrasis pipe for text pre-processing
def ekphrasis_pipe(sentence):
    cleaned_sentence = " ".join(text_processor.pre_process_doc(sentence))
    return cleaned_sentence

# Training set
text_array = text_array.apply(lambda text: ekphrasis_pipe(text))
print("Training set completed.......")
#Validation set
text_array_val = text_array_val.apply(lambda text: ekphrasis_pipe(text))
print("Test set completed.......")

In [ ]:
print_text(text_array,0,10)
print("************************************************************************")
print_text(text_array_val,0,10)

In [ ]:
# Removing unnecessary punctuations
PUNCT_TO_REMOVE = "\"$%&'()+,-./;=[\]^_`{|}~"
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

# Training set
text_array = text_array.apply(lambda text: remove_punctuation(text))
print_text(text_array,0,10)

print("********************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: remove_punctuation(text))
print_text(text_array_val,0,10)

In [ ]:
# Finding length of longest array
maxLen = len(max(text_array,key = lambda text: len(text.split(" "))).split(" "))
print(maxLen)

In [ ]:
u = lambda text: len(text.split(" "))
sentence_lengths = []
for x in text_array:
    sentence_lengths.append(u(x))
print(sorted(sentence_lengths)[-800:])
print(len(sentence_lengths))

In [34]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map, words

In [35]:
# We need word_to_index and index_to_word for creating the embedding layer.
word_to_index, index_to_word, word_to_vec_map, vocab = read_glove_vecs('/content/drive/My Drive/Semeval 2017/twitter.100d.txt')

In [ ]:
print(len(word_to_index))
print(len(text_array))

In [ ]:
# Test function for array dimensionality
u = 0
for word, index in word_to_index.items():
        enc = word_to_vec_map[word]
        # Handling error in glove 
        if not (np.shape(enc)[0] != (100,)):
            u = u + 1
print(u)

In [ ]:
# Test function for counting the missing vocabulary count
vocab_missing_count = 0
vocab_count = 0
for text in text_array:
    local = 0
    for word in text.split(" "):
        vocab_count += 1
        if not word == "":
            if not word in word_to_vec_map:
                print(word)
                vocab_missing_count += 1
                local += 1
    if not (local==0):
        print(local)
        print()

#Printing overall details
print("Total vocabulary count: "+ str(vocab_count))
print("Missing vocabulary count: "+ str(vocab_missing_count))

In [ ]:
# Count of each label in dataset
from collections import Counter

# Printing training set counts for analysis
print("Elements: ",set(labels))
print("Length: ",len(labels))
print(Counter(labels))

print("**************************************************************************")

# Printing validation set counts for analysis
print("Elements: ",set(labels_val))
print("Length: ",len(labels_val))
print(Counter(labels_val))

In [40]:
Y = []
Y_val = []

# Training set    
for i in range(0,len(labels)):
    if(labels[i] == 'neutral'):
        Y.append(0)
    if(labels[i] == 'positive'):
        Y.append(1)
    if(labels[i] == 'negative'):
        Y.append(2)


# Validation set
for i in range(0,len(labels_val)):
    if(labels_val[i] == 'neutral'):
        Y_val.append(0)
    if(labels_val[i] == 'positive'):
        Y_val.append(1)
    if(labels_val[i] == 'negative'):
        Y_val.append(2)

In [ ]:
print(len(Y),len(Y_val))

In [ ]:
print(Counter(Y))
print(Counter(Y_val))

In [ ]:
# Testing the conversion into integers
for i in range(310,320):
    print(text_array_val[i])
    print(labels_val[i],Y_val[i])

In [ ]:
# Verifying train set 
X = np.asarray(list(text_array))
Y = np.asarray(list(Y))
labels = np.asarray(list(labels))
print(type(X))
print(type(Y))
print(type(labels))
print(np.shape(X),np.shape(Y),np.shape(labels))

# Verifying validation set
X_val = np.asarray(list(text_array_val))
Y_val = np.asarray(list(Y_val))
labels_val = np.asarray(list(labels_val))
print(type(X_val))
print(type(Y_val))
print(type(labels_val))
print(np.shape(X_val),np.shape(Y_val),np.shape(labels_val))

<h2>Shuffling training and validation data</h2>

In [45]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
X, Y, labels = shuffle(X, Y, labels, random_state=44)
print("Size of training data: " + str(np.shape(X)))
print("Size of test data: " + str(np.shape(X_val)))

In [ ]:
print(Counter(Y))
print(Counter(labels))

In [ ]:
# Testing after shuffling
for i in range(310,320):
    print(text_array_val[i])
    print(labels_val[i],Y_val[i])

In [ ]:
index = 824
print(X[index])
print(labels[index])
print(Y[index])

In [ ]:
print(type(X))
print(type(Y))
print(np.shape(X),np.shape(Y),np.shape(labels))
print(np.shape(X_val),np.shape(Y_val),np.shape(labels_val))

In [51]:
# Converting to one hot vectors
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)] #u[Y] helps to index each element of Y index at u. U here is a class array
    return Y

In [ ]:
Y_oh_train = convert_to_one_hot(np.array(Y), C = 3)
Y_oh_val = convert_to_one_hot(np.array(Y_val), C = 3)

print(np.shape(Y_oh_train))
index = 310
print(labels[index], Y[index], "is converted into one hot", Y_oh_train[index])

<h2>Tensorflow Model</h2>

In [ ]:
import tensorflow as tf
import keras
import keras.backend as K
from keras.layers import LeakyReLU
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Concatenate, Bidirectional, Add, RepeatVector, Dot, BatchNormalization, Conv2D, Flatten, GaussianNoise, GRU
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.utils import plot_model
from keras_self_attention import SeqSelfAttention

In [ ]:
print(tf.__version__)
print(keras.__version__)

In [55]:
# Function to conver sentence to indices using the glove vocab
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.

    """
    
    m = X.shape[0]                                 
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                              
        sentence_words =(X[i].lower()).split(" ")
        j = 0        
        
        for w in sentence_words:
            if j==max_len:
                break
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            if w in word_to_index:
                X_indices[i, j] = word_to_index[w]
                # Increment j to j + 1
                j = j+1
            
    
    return X_indices

In [ ]:
# Test for sentence_to_indices
print("Text: " + text_array[0])
test = sentences_to_indices(text_array[:2],word_to_index,15)
print(test)

In [57]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    """
    
    vocab_len = len(word_to_index) + 1                  
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of GloVe word vectors (= 100)
    
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        enc = word_to_vec_map[word]
        # Handling error in glove 
        if not (np.shape(enc)[0] == emb_dim):
            temp = np.zeros((emb_dim,))
            temp[:len(enc)] = word_to_vec_map[word]
            enc = temp
        # Handling of error complete
        emb_matrix[index] = enc

    embedding_layer = Embedding(vocab_len, emb_dim, mask_zero = True, trainable = False)
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [ ]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

In [60]:
def softmax(x, axis=1):
    """Softmax activation function."""
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')
        


In [61]:
# K.clear_session()

In [106]:
def Offense_classifier(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the model's graph.
    
    """

    sentence_indices = Input(shape = input_shape, dtype='int32')
    
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)
    embeddings = GaussianNoise(0.2)(embeddings)
    embeddings = Dropout(0.3)(embeddings)

    # Propagate the embeddings through an LSTM layer with 150-dimensional hidden state
    lstm_one = Bidirectional(LSTM(150, return_sequences=True, recurrent_dropout=0.25, dropout=0.2)) 
    X = lstm_one(embeddings)
    X = Dropout(0.2)(X)

    lstm_two = Bidirectional(LSTM(150, return_sequences=True, recurrent_dropout=0.25, dropout=0.2)) 
    X = lstm_two(X)
    X = Dropout(0.2)(X)

    # *************Attention*******************
    X = SeqSelfAttention(attention_activation='elu')(X)
    # ****************Attention*******************

    post_activation_GRU_cell = GRU(64, return_sequences = False, recurrent_dropout=0.25, dropout=0.2)
    X = post_activation_GRU_cell(X)

    X = Dense(32,activation='elu',kernel_regularizer=keras.regularizers.l2(0.0001))(X)
    X = BatchNormalization(momentum=0.99, epsilon=0.001, center=True, scale=True)(X)

    X = Dense(3,activation='elu',kernel_regularizer=keras.regularizers.l2(0.0001))(X)

    
    # Add a sigmoid activation
    X = Activation('sigmoid')(X)
    
    model = Model(inputs=[sentence_indices], outputs=[X])
    
    
    return model

In [ ]:
model = Offense_classifier((40,), word_to_vec_map, word_to_index)
model.summary()

In [108]:
model.compile(loss=['categorical_crossentropy'], optimizer="adam", metrics=['acc'])

In [109]:
X_train_indices = sentences_to_indices(X, word_to_index, 40)
print(np.shape(X_train_indices))
X_val_indices = sentences_to_indices(X_val, word_to_index, 40)
print(np.shape(X_val_indices))

(50132, 40)
(12284, 40)


In [110]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='/content/semeval_2017_task4.{epoch:03d}.h5',
                                 monitor='val_acc',
                                 verbose = 1,
                                 save_weights_only=True,
                                 period=1)

In [ ]:
# Counts of each class in training data. 
c = Counter(Y)
print(c)
print(c.keys())
neutral = c[0]
pos = c[1]
neg = c[2]
total = pos+neg+neutral
print(neutral,pos,neg,total)

In [ ]:
# Class weights
maxi = max(pos,neg,neutral)
weight_for_0 = (maxi / (maxi+neutral))
weight_for_1 = (maxi / (maxi+pos))
weight_for_2 = (maxi / (maxi+neg))

class_weight = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))
print('Weight for class 2: {:.2f}'.format(weight_for_2))

In [ ]:
history = model.fit(
    [X_train_indices],
    [Y_oh_train],
    batch_size = 128,
    epochs = 9,
    validation_data = ([X_val_indices],[Y_oh_val]),
    callbacks = [checkpoint],
    class_weight = [class_weight]
)

In [ ]:
history = history
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train-sig', 'Test-sig'], loc='lower right')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='lower right')
plt.show()

In [115]:
# plot_model(model, to_file="model.png", show_shapes=True, show_layer_names=False)

In [117]:
answer = model.predict([X_val_indices])

In [ ]:
print(X_val[0])
print(Y_oh_val[0])
print(labels_val[0])
print("******************************************")
print(len(answer),len(answer))

In [ ]:
count_sl = 0
count_pos = 0
count_not = 0
pred = []
text = df_val[2]

temp = 0
for i in range(0,len(X_val)):
    num = np.argmax(answer[i])
    pred.append(num)
    if(num != Y_val[i] and Y_val[i] == 1):
        print(answer[i])
        print(labels_val[i])
        print(text[i])
        lol = ""
        temp += 1
        if(num == 0):
            lol = "neutral"
            count_not += 1
        if(num == 1):
            lol = "positive"
            count_sl += 1
        if(num == 2):
            lol = "negative"
            count_pos += 1
        print("Predicted: " + lol)
        print()

print(temp)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y_val, predictions=pred)
con_mat = tf.Session().run(con_mat)
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
print(con_mat)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat, annot=True,cmap=plt.cm.Spectral,fmt='d',xticklabels=["Neutral","Positive","Negative"], yticklabels=["Neutral","Positive","Negative"])
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
from sklearn.metrics import f1_score
f1_score(Y_val, pred, average='macro')

In [ ]:
from sklearn.metrics import recall_score
recall_score(Y_val, pred, average='weighted')

In [ ]:
from sklearn.metrics import classification_report
target_names = ['Neutral', 'Positive', 'Negative']
print(classification_report(Y_val, pred, digits=3))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_val, pred, normalize=True)

In [ ]:
pip install plotly==4.5.4

In [104]:
import plotly
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
flag = []
count = 0

positive = []
negative = []
neutral = []


for i in range(0,len(pred)):
    count = count + 1
    neutral.append(answer[i][0])
    positive.append(answer[i][1])
    negative.append(answer[i][2])


print(count)

In [106]:
pred_colour = []
for i in range(0,len(pred)):
    if pred[i] == 0:
      pred_colour.append("Neutral")
    if pred[i] == 1:
      pred_colour.append("Positive")
    if pred[i] == 2:
      pred_colour.append("Negative")

test_df = pd.DataFrame({'positive':positive, 'negative':negative, 'neutral':neutral, 'Prediction':pred_colour})

fig = px.scatter_3d(test_df, x='positive', y='negative', z='neutral', color='Prediction')
fig.update_traces(
    marker={
        'size': 0.7,
        'opacity': 1,
        'colorscale' : 'viridis',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'})
fig.update_layout(width = 700)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

<h2>Clustering</h2>

In [109]:
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

<h3>Prediction layer</h3>

In [114]:
kmeans = KMeans(n_clusters=3, random_state=4).fit(answer)
y_kmeans_3 = kmeans.predict(answer)

In [115]:
# Correcting the labels of clustering as per the original prediction values
for i in range(0,len(y_kmeans_3)):
    if(y_kmeans_3[i] == 0):
      y_kmeans_3[i] = 2
    elif(y_kmeans_3[i] == 1):
      y_kmeans_3[i] = 0
    else:
      y_kmeans_3[i] = 1

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y_val, predictions=y_kmeans_3)
con_mat = tf.Session().run(con_mat)
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
print(con_mat)

In [ ]:
from sklearn.metrics import f1_score
f1_score(Y_val, y_kmeans_3, average='macro')

In [ ]:
from sklearn.metrics import recall_score
recall_score(Y_val, y_kmeans_3, average='weighted')

In [ ]:
from sklearn.metrics import classification_report
target_names = ['Neutral', 'Positive', 'Negative']
print(classification_report(Y_val, y_kmeans_3, digits=3, target_names=target_names))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_val, y_kmeans_3, normalize=True)

<h5>SVNS Calculation</h5>

In [124]:
centers_3 = kmeans.cluster_centers_

In [125]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

In [ ]:
svns_neu = []
for i in range(0,len(Y_val)):
    neu = cosine(answer[i], centers_3[1])/2
    svns_neu.append(1-neu)
print(len(svns_neu))

In [ ]:
svns_pos = []
for i in range(0,len(Y_val)):
    pos = cosine(answer[i], centers_3[2])/2
    svns_pos.append(1-pos)
print(len(svns_pos))

In [ ]:
svns_neg = []
for i in range(0,len(Y_val)):
    neg = cosine(answer[i], centers_3[0])/2
    svns_neg.append(1-neg)
print(len(svns_neg))

<h5>SVNS plot from prediction layer Clustering</h5>

In [129]:
pred_colour = []
for i in range(0,len(pred)):
    if y_kmeans_3[i] == 0:
      pred_colour.append("Neutral")
    if y_kmeans_3[i] == 1:
      pred_colour.append("Positive")
    if y_kmeans_3[i] == 2:
      pred_colour.append("Negative")

test_df = pd.DataFrame({'SVNS Positive':svns_pos, 'SVNS Negative':svns_neg, 'SVNS Neutral':svns_neu, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='SVNS Positive', y='SVNS Negative', z='SVNS Neutral', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'viridis',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'})
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

<h3>Batch Normalisation Layer</h3>

In [ ]:
model.layers[-3]

In [131]:
cl_model = Model(model.input, model.layers[-3].output)
cl_32 = cl_model.predict([X_val_indices])

In [135]:
kmeans = KMeans(n_clusters=3, random_state=4).fit(cl_32)
y_kmeans_batchnorm = kmeans.predict(cl_32)

In [136]:
for i in range(0,len(y_kmeans_batchnorm)):
    if(y_kmeans_batchnorm[i] == 0):
      y_kmeans_batchnorm[i] = 2
    elif(y_kmeans_batchnorm[i] == 1):
      y_kmeans_batchnorm[i] = 1
    else:
      y_kmeans_batchnorm[i] = 0

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y_val, predictions=y_kmeans_batchnorm)
con_mat = tf.Session().run(con_mat)
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
print(con_mat)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['Neutral', 'Positive', 'Negative']
print(classification_report(Y_val, y_kmeans_batchnorm, digits=3, target_names=target_names))

<h5>SVNS Calculation</h5>

In [139]:
centers_batchnorm = kmeans.cluster_centers_

In [ ]:
svns_neu = []
for i in range(0,len(Y_val)):
    neu = cosine(cl_32[i], centers_batchnorm[2])/2
    svns_neu.append(1-neu)
print(len(svns_neu))

In [ ]:
svns_pos = []
for i in range(0,len(Y_val)):
    pos = cosine(cl_32[i], centers_batchnorm[1])/2
    svns_pos.append(1-pos)
print(len(svns_pos))

In [ ]:
svns_neg = []
for i in range(0,len(Y_val)):
    neg = cosine(cl_32[i], centers_batchnorm[0])/2
    svns_neg.append(1-neg)
print(len(svns_neg))


<h5>SVNS plot from Batch Normalisation layer Clustering</h5>

In [143]:
pred_colour = []
for i in range(0,len(pred)):
    if y_kmeans_batchnorm[i] == 0:
      pred_colour.append("Neutral")
    if y_kmeans_batchnorm[i] == 1:
      pred_colour.append("Positive")
    if y_kmeans_batchnorm[i] == 2:
      pred_colour.append("Negative")

test_df = pd.DataFrame({'SVNS Positive':svns_pos, 'SVNS Negative':svns_neg, 'SVNS Neutral':svns_neu, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='SVNS Positive', y='SVNS Negative', z='SVNS Neutral', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'viridis',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'})
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

<h3>GRU Layer</h3>

In [ ]:
model.layers[-5]

In [145]:
cl_model = Model(model.input, model.layers[-5].output)
cl_32 = cl_model.predict([X_val_indices])

In [156]:
kmeans = KMeans(n_clusters=3, random_state=4).fit(cl_32)
y_kmeans_gru = kmeans.predict(cl_32)

In [157]:
for i in range(0,len(y_kmeans_gru)):
    if(y_kmeans_gru[i] == 0):
      y_kmeans_gru[i] = 2
    elif(y_kmeans_gru[i] == 1):
      y_kmeans_gru[i] = 0
    else:
      y_kmeans_gru[i] = 1

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y_val, predictions=y_kmeans_gru)
con_mat = tf.Session().run(con_mat)
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
print(con_mat)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['Neutral', 'Positive', 'Negative']
print(classification_report(Y_val, y_kmeans_gru, digits=3, target_names=target_names))

<h5>SVNS Calculation</h5>

In [160]:
centers_gru = kmeans.cluster_centers_

In [ ]:
svns_neu = []
for i in range(0,len(Y_val)):
    neu = cosine(cl_32[i], centers_gru[1])/2
    svns_neu.append(1-neu)
print(len(svns_neu))

In [ ]:
svns_pos = []
for i in range(0,len(Y_val)):
    pos = cosine(cl_32[i], centers_gru[2])/2
    svns_pos.append(1-pos)
print(len(svns_pos))

In [ ]:
svns_neg = []
for i in range(0,len(Y_val)):
    neg = cosine(cl_32[i], centers_gru[0])/2
    svns_neg.append(1-neg)
print(len(svns_neg))

<h5>SVNS plot from gru layer Clustering</h5>

In [164]:
pred_colour = []
for i in range(0,len(pred)):
    if y_kmeans_gru[i] == 0:
      pred_colour.append("Neutral")
    if y_kmeans_gru[i] == 1:
      pred_colour.append("Positive")
    if y_kmeans_gru[i] == 2:
      pred_colour.append("Negative")

test_df = pd.DataFrame({'SVNS Positive':svns_pos, 'SVNS Negative':svns_neg, 'SVNS Neutral':svns_neu, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='SVNS Positive', y='SVNS Negative', z='SVNS Neutral', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'viridis',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'})
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))